# Import

In [1]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
# import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torch import nn
from tqdm import tqdm

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)
device = torch.device('cuda:0')

# Config

# Data Preparation

In [2]:
# video_labels = pd.read_csv('/home/thinh/nfl/train_labels.csv')
# video_labels = video_labels[video_labels['frame'] != 0].reset_index(drop=True)
# video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str) + '.png'

In [3]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

# Dataset

In [4]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

# Define Validation Image Augmentations

In [5]:
def get_inference_transforms():
    return A.Compose([
            A.RandomResizedCrop(512, 512),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# Model

In [6]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

# Main Loop

In [7]:
def inference_one_model(model, data_loader):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [8]:
DATA_ROOT_PATH = '/home/thinh/nfl/train_images/'
test = pd.DataFrame()
test['image_id'] = list(os.listdir(f'{DATA_ROOT_PATH}'))
video_valid = ['57583_000082', '57586_004152', '57911_000147', '57997_003691', '57680_002206', '58095_004022', '57906_000718', '58005_001254', '57679_003316', '58103_003494', '57998_002181', '58048_000086']
test = test[test['image_id'].apply(lambda x: "_".join(x.split('_')[:2])).isin(video_valid)]

test_ds = CassavaDataset(test, f'{DATA_ROOT_PATH}', transforms=get_inference_transforms(), output_label=False)
# image_ids=np.array([path.split('/')[-1] for path in glob(f'{DATA_ROOT_PATH}/*.png')]),
tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size=16,
            num_workers=8,
            shuffle=False,
            pin_memory=False,
        )

model_imgs = ['/home/thinh/nfl/frame-models/tf_efficientnet_b4_ns_512_fold_0_0',
              '/home/thinh/nfl/frame-models/tf_efficientnet_b4_ns_512_fold_1_0',
              '/home/thinh/nfl/frame-models/tf_efficientnet_b4_ns_512_fold_2_0',
              '/home/thinh/nfl/frame-models/tf_efficientnet_b4_ns_512_fold_3_0',
              '/home/thinh/nfl/frame-models/tf_efficientnet_b4_ns_512_fold_4_0',
             ]

tst_preds = []
for model_img in model_imgs:
    model = CassvaImgClassifier('tf_efficientnet_b4_ns', 2).to(device)
    model.load_state_dict(torch.load(model_img, map_location='cuda:0'))
    
    with torch.no_grad():
        for _ in range(3):
            tst_preds += [inference_one_model(model, tst_loader)]
            
tst_preds = np.mean(tst_preds, axis=0) 

100%|██████████| 653/653 [03:13<00:00,  3.37it/s]


In [9]:
# test = pd.DataFrame()
# test['image_id'] = list(os.listdir(f'{DATA_ROOT_PATH}'))
test['label'] = np.argmax(tst_preds, axis=1)
test = test[test.label == 1]
test['frame'] = test.image_id.str.split('_').str[3].str.replace('.png','').astype(int)
test['video'] = test.image_id.str.rsplit('_',1).str[0] + '.mp4'
test = test.drop(columns=['image_id'])
print(test.shape)
test.head()

(3326, 3)


,label,frame,video
14,1,141,57680_002206_Sideline.mp4
45,1,158,57679_003316_Endzone.mp4
68,1,120,57583_000082_Endzone.mp4
69,1,42,57586_004152_Sideline.mp4
104,1,136,57997_003691_Endzone.mp4


In [10]:
test.to_csv('frame_impact_512.csv', index=False)